### Modelagem do Problema do Fluxo de Caixa
Prof. Alexandre Salles da Cunha

In [21]:
from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition
import numpy as np
import math

In [22]:
m = 10 # no. de titulos
n = 6  # no. de meses
J = list(range(0,m)) # conjunto de titulos
I = list(range(0,n)) # conjunto de períodos de tempo
p = [109.0, 94.8, 99.5, 93.1, 97.2, 92.9, 110.0, 104.0, 102, 95.2]
y   = [100, 200, 800, 100, 800, 1200]
c = [[10.0, 7.0, 8.0, 6.0, 7.0, 5.0, 10.0, 8.0, 7.0, 100],
                  [10.0, 7.0, 8.0, 6.0, 7.0, 5.0, 10.0, 8.0, 107.0,0.0],
                  [10.0, 7.0, 8.0, 6.0, 7.0, 5.0, 110.0, 108.0, 0.0, 0.0],
                  [10.0, 7.0, 8.0, 6.0, 7.0, 105.0, 0.0, 0.0, 0.0, 0.0],
                  [10.0, 7.0, 8.0, 106.0, 107.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                  [110.0, 107.0, 108.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
                  

In [23]:
def RestricaoNecessidadeCaixa(Model,J,i,c,y):
    expressao = sum(Model.x[j] * c[i][j] for j in J) - y[i]
    return expressao >= 0.0

In [24]:
Caixa = ConcreteModel()
# x representa a quantidade de titulos do tipo j adquirida no periodo i
Caixa.x = Var(J,within = NonNegativeReals)
Caixa.obj = Objective(expr = sum(p[j] * Caixa.x[j] for j in J), sense = minimize)
Caixa.atendimento = ConstraintList()
for i in I:
    expressao = RestricaoNecessidadeCaixa(Caixa,J,i,c,y)
    Caixa.atendimento.add(expressao)

In [40]:
#solver = SolverFactory('xpress_direct') ## pacote comercial de alto desempenho
solver = SolverFactory('glpk') ## codigo shareware, gratuito de prog linear inteira
#solver = SolverFactory('ipopt') ## codigo de programaçao não linear, gratuito
solver.solve(Caixa,tee = True)
solver.solve(Caixa)


custo = value(Caixa.obj)
print("Custo total dos titulos = %5.2f" % custo)
for j in J:
    if value(Caixa.x[j]) > 0.0:
        print('Titulo',j,"quantidade, x = %5.2f" % value(Caixa.x[j]))



GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmpjgb0og1o.glpk.raw --wglp /tmp/tmpk98ojvsj.glpk.glp --cpxlp
 /tmp/tmp4lsil960.pyomo.lp
Reading problem data from '/tmp/tmp4lsil960.pyomo.lp'...
7 rows, 11 columns, 42 non-zeros
91 lines were read
Writing problem data to '/tmp/tmpk98ojvsj.glpk.glp'...
80 lines were written
GLPK Simplex Optimizer, v4.65
7 rows, 11 columns, 42 non-zeros
Preprocessing...
6 rows, 10 columns, 41 non-zeros
Scaling...
 A: min|aij| =  5.000e+00  max|aij| =  1.100e+02  ratio =  2.200e+01
GM: min|aij| =  2.345e-01  max|aij| =  4.264e+00  ratio =  1.818e+01
EQ: min|aij| =  5.940e-02  max|aij| =  1.000e+00  ratio =  1.684e+01
Constructing initial basis...
Size of triangular part is 6
      0: obj =   0.000000000e+00 inf =   2.422e+03 (6)
      5: obj =   2.759256000e+03 inf =   0.000e+00 (0)
*     6: obj =   2.381138830e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (42043 byt